In [ ]:
from kg_extract.doc_handle import RuDocHandle

document = RuDocHandle('test.docx')
data = document.get_docx_structure_v2()
catalog = document.get_catalog()
# document.recovery_docx()

In [ ]:
style = []
for temp in data['content']:
    style.append(temp['style'])
set(style)

In [ ]:
from kg_extract.doc_handle import BasePptxHandle
document = BasePptxHandle('test.pptx')
data = document.get_pptx_structure()

In [ ]:
len(data)

In [ ]:
from docx import Document

doc = Document('test.docx')
count = 0
for par in doc.paragraphs:
    style = par.style.name
    content = par.text.strip()
    print(style, content)

In [ ]:
from kg_extract.ner import RuNer

document = RuNer('en_core_web_sm', 'test.docx')

In [ ]:
data = document.get_sentences_ner()

In [ ]:
proper_nouns = list(pd.read_csv('proper_nouns.csv')['proper_nouns_name'].values)
proper_nouns

In [ ]:
temp = 'On the downlink path, the Digital Subsystem DU receives information through the Transport Network interface and uses Baseband signal processing to encode the data as IQ modulated digital data to meet the requirements of the antenna interface (AI)'.split(' ')
set(proper_nouns).intersection(set(temp))

In [ ]:
data.to_csv('./man.csv',index=False)

In [ ]:
from kg_process.es import Ru_es

es = Ru_es("http://localhost:7474",auth=("neo4j","test"))
es.insert()

In [ ]:
es.insert()

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

doc = {
    's': '',
    'p': '',
    'o': '',
    'key_words': [],
    'timestamp': datetime.now(),
}
# res = es.index(index="test-index", id=1, body=doc)
# print(res['result'])

In [ ]:
graph_data = pd.read_csv('man.csv')
for i in range(0, len(graph_data)):
    doc['s'] = graph_data['s'][i]
    doc['p'] = graph_data['p'][i]
    doc['o'] = graph_data['o'][i]
    doc['key_words'] = eval(graph_data['key_words'][i])
    doc['timestamp'] = datetime.now()
    es.index(index="doc", doc_type='Radio Application Architecture Description', body=doc)

In [ ]:
# res = es.get(index="test-index", id=1)
# print(res['_source'])

# es.indices.refresh(index="test-index")

res = es.search(index="doc", doc_type='Radio Application Architecture Description', 
                body={
                        "query": {
                            "match": {
                                "o": {      
                                    "query":    "RBS 3GPP",
                                    "operator": "and"
                                }
                            }
                        }
                    }
               )
# print("Got %d Hits:" % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
from PIL import Image
import cv2 

image = cv2.imread('image3.jpg',0)
image = cv2.Laplacian(image,cv2.CV_64F)
# plt.hist(img.ravel(),256,[0,256]); plt.show()
# clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4,4))
# image = clahe.apply(img)

(origH, origW) = image.shape[:2]
(newW, newH) = (int(origW*1.5), int(origH*1.5))
image = cv2.resize(image, (newW, newH))

# ret,image = cv2.threshold(image,40,255,cv2.THRESH_BINARY)

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# dst = 255 - gray
# cv2.imshow('dst', dst)

plt.hist(image.ravel(),256,[0,256]); plt.show()

plt.rcParams['figure.figsize'] = (newW/100, newH/100) # 设置figure_size尺寸
plt.rcParams['figure.dpi'] = 100 #分辨率

plt.imshow(image)

In [ ]:
img

In [ ]:
import matplotlib.pyplot as plt  
from PIL import Image
import cv2

image = cv2.imread('image3.jpg')

# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

(origH, origW) = image.shape[:2]
(newW, newH) = (int(origW*1.5), int(origH*1.5))
image = cv2.resize(image, (newW, newH))

plt.rcParams['figure.figsize'] = (newW/100, newH/100) # 设置figure_size尺寸
plt.rcParams['figure.dpi'] = 100 #分辨率

plt.imshow(image)

In [ ]:
temp = 'class Tx'
temp.split(':')

In [ ]:
include = []
namespace = []
with open('txBackoffCtrl.h', 'r') as f:
    for line in f.readlines():
        line = line.strip().replace('  ', ' ')
        if not line.startswith('#include'):
            continue
        
        if line.startswith('#include'):
            include.append(line.split(' ')[1])
            
        if line.startswith('namespace'):
            namespace.append(line.split(' ')[1])
            
        if line.startswith('class'):
            

In [ ]:
from clang.cindex import Config
from clang.cindex import Index

libclangPath = r'C:\Program Files\LLVM\bin'
Config.set_library_path(libclangPath)

In [1]:
from clang.cindex import Config
from clang.cindex import TypeKind
from clang.cindex import CursorKind
from clang.cindex import Index
#首先需要导入需要的类型包。
#clang的python绑定的具体配置方法看博主的另一篇博客clang的python接口(-)
# libclangPath = '/usr/lib/llvm-4.0/lib/libclang-4.0.so.1'

def method_definitions(cursor):
    for i in cursor.walk_preorder():
        if i.kind != CursorKind.CXX_METHOD:
            continue
        if not i.is_definition():
            continue
        yield i

def extract_definition(cursor):
    filename = cursor.location.file.name
    with open(filename, 'r') as fh:
        contents = fh.read()
    return contents[cursor.extent.start.offset: cursor.extent.end.offset]

libclangPath = r'C:\Program Files\LLVM\bin\libclang.dll'
# if Config.loaded==True:
#     pass
# else:
Config.set_library_file(libclangPath)
#libclangPath是libclang.so的具体位置。
index = Index.create()
tu = index.parse('txBackoffCtrl.h')

# defns = method_definitions(tu.cursor)
# for defn in defns:
#     print(extract_definition(defn))

In [5]:
temp = tu
for i in temp.cursor.walk_preorder():
    print(i.)

In [10]:
with open('txBackoffCtrl.h', 'r') as fh:
    contents = fh.read()
data = contents[tu.cursor.extent.start.offset: tu.cursor.extent.end.offset]

In [11]:
data

'/******************************************************************************\n ************************* Copyright ERICSSON AB 2014 **************************\n *******************************************************************************\n *\n * The Copyright to the computer programs herein is the property of ERICSSON AB.\n * The programs may be used and/or copied only with the written permission from\n * ERICSSON AB or in accordance with the terms and conditions stipulated in\n * the agreement/contract under which the programs have been supplied.\n *\n *******************************************************************************\n */\n#ifndef TXBACKOFFCTRL_H_\n#define TXBACKOFFCTRL_H_\n\n#include <set>\n#include "overTempSrvIf.h"\n#include "overPowerSrvIf.h"\n#include <osetypes.h>\n#include <ruError.h>\n#include "txBackoffCtrlIf.h"\n#include "txServiceContainer.h"\n#include "serviceCommonData.h"\n#include "serviceLocatorIf.h"\n#include "instanceToken.h"\n#include "activeObjec

In [ ]:
import clang.cindex
from clang.cindex import *



libclangPath = "C:\\Program Files\\LLVM\\bin\\libclang.dll"
# if Config.loaded==True:
#     pass
# else:
Config.set_library_file(libclangPath)

idx = Index.create()
tu = idx.parse('txBackoffCtrl.h')
# defns = method_definitions(tu.cursor)
# for defn in defns:
#     print(extract_definition(defn))

In [ ]:
Config.get_filename()

In [ ]:
temp